In [ ]:
import dotenv
import os
dotenv.load_dotenv() # For OpenRouter API key

In [ ]:
CSV_FILE = ""

In [ ]:
# Import RAG results
import pandas as pd

# Import csv file as dataframe
df = pd.read_csv(CSV_FILE, sep=";") # Assumes that CSV file is in the same directory as this script

In [ ]:
from haystack.components.builders import PromptBuilder

template = """
You will be provided with three pieces of text. One will be a question, one will be the Generated Answer that has been generated by a question answering system and the other will be the correct answer. You will be asked to determine if the generated answer is correct or not. Take the question into account. 
If the generated answer is correct, you will respond with just the digit 1. If the generated answer is incorrect, you will respond with just the digit 0. If you are unsure, you will respond with the digit 2. 
DO NOT respond with anything other than those digits. DO NOT add any additional comments or information.

Question = {{question}}
Generated Answer: {{generated_answer}}
Correct Answer: {{correct_answer}}
"""

prompt_builder = PromptBuilder(template=template)

In [ ]:
from haystack.components.generators import OpenAIGenerator
from haystack.utils import Secret

MODEL_NAME = "meta-llama/llama-3-70b-instruct:nitro"

generator = OpenAIGenerator(model=MODEL_NAME,
                            api_key=Secret.from_env_var("OPENROUTER_API_KEY"),
                            api_base_url="https://openrouter.ai/api/v1",
                            generation_kwargs={
                              "temperature": 0.0,
                            })

In [ ]:
from haystack import Pipeline

llm_pipeline = Pipeline()
# Add components to your pipeline
llm_pipeline.add_component("prompt_builder", prompt_builder)
llm_pipeline.add_component("llm", generator)

# Now, connect the components to each other
llm_pipeline.connect("prompt_builder", "llm")

In [ ]:
# Convert overlap column to a string column
df["overlap"] = df["overlap"].astype(str)
# Make overlap column contain only ""
df["overlap"] = ""

In [ ]:
# Get the total number of rows in the DataFrame for the progress bar
total_rows = len(df[(df["overlap"] == "") | (df["overlap"] == "ERROR") | (df["overlap"].str.contains("<!DOCTYPE html>", na=False))])

In [ ]:
from tqdm import tqdm

# Create a progress bar
with tqdm(total=total_rows, desc="Processing rows", dynamic_ncols=True) as pbar:
    # Iterate over each row
    for index, row in df.iterrows():
        if row["overlap"] != "" and row["overlap"] != "ERROR" and not "<!DOCTYPE html>" in row["overlap"]:
            continue
        
        # Get the question and the context
        question = row["questions"]
        generated_result = row["result"]
        correct_result = row["correct_result"]
        # Run the pipeline
        try:
            result = llm_pipeline.run({"prompt_builder": {"question": question, "generated_answer": generated_result, "correct_answer": correct_result}})["llm"]["replies"][0]
        except Exception as e:
            print(str(e))
            result = "ERROR"
        # Add to df
        df.at[index, "overlap"] = result

        # Update the progress bar
        pbar.update()

In [ ]:
# Save the results to a new file
df.to_csv(f"eval_{CSV_FILE}", index=False)